In [1]:
import pandas
import os

data_path = 'perf-data'
devices = ['3b+', '3b.']
runtimes = ['go', 'js', 'py','native']

In [2]:
flat_files = []
iterations = os.listdir(data_path)

for it in iterations:
    it_files = os.listdir(f"{data_path}/{it}")
    for it_file in it_files:
        flat_files.append(f"{data_path}/{it}/{it_file}")

In [3]:
runtime_device_files_total = {}
for device in devices:
    runtime_device_files_total[device] = {}
    device_files = list(filter(lambda file: device in file, flat_files))
    for runtime in runtimes:
        runtime_device_files = list(filter(lambda file: runtime in file, device_files))
        runtime_device_files_total[device][runtime] = runtime_device_files

In [4]:
def getConcatData(files):    
    runtime_data = pandas.DataFrame()
    for file in files:
        data = pandas.read_csv(file)
        runtime_data = runtime_data.append(data, ignore_index=True)
    return runtime_data

runtime_device_data = {}
for device in runtime_device_files_total.keys():
    runtime_device_data[device] = {}
    for runtime in runtime_device_files_total[device].keys():
        runtime_device_data[device][runtime] = getConcatData(runtime_device_files_total[device][runtime])

In [5]:
def pretty_print_data(data, unit='ns'):
    for device in data.keys():
        print(f"{device}:")
        for runtime in data[device].keys():
            converted_data = data[device][runtime]
            if unit == 'µs':
                converted_data = converted_data / 1000
            if unit == 'ms':
                converted_data = converted_data / 1000000
            if unit == 's':
                converted_data = converted_data / 1000000000
            print(f"\t{runtime}: {converted_data}")

## Visualize time per execution

In [12]:
import matplotlib.pyplot as plt
cols = len(iterations)
rows = len(runtimes)
cols_headers = ['Iteration {}'.format(col) for col in range(cols)]
rows_headers = runtimes[::]
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 8))
device = '3b+'
for idx_runtime, runtime in enumerate(runtimes):
    for idx_it, it in enumerate(iterations):
        date = pandas.read_csv(runtime_device_files_total[device][runtime][idx_it])
        axes[idx_runtime, idx_it].scatter(date.index.values.tolist(), date['time (ns)'])

for ax, col in zip(axes[0], cols_headers):
    ax.set_title(col)

for ax, row in zip(axes[:,0], rows_headers):
    ax.set_ylabel(row, rotation=0, size='large')
fig.tight_layout()
plt.show()

TypeError: 'int' object is not iterable

## Data time means (ns)

In [39]:
time_field = 'time (ns)'
runtime_device_data_means = {}
for device in runtime_device_data.keys():
    runtime_device_data_means[device] = {}
    for runtime in runtime_device_data[device].keys():
        runtime_device_data_means[device][runtime] = runtime_device_data[device][runtime][time_field].mean()
pretty_print_data(runtime_device_data_means, 'ns')

delta_means_rel = {}
for device in runtime_device_data_means.keys():
    delta_means_rel[device] = {}
    for runtime in runtime_device_data_means[device].keys():
        if runtime == 'native':
            continue
        delta_means_rel[device][runtime] = runtime_device_data_means[device][runtime] / runtime_device_data_means[device]['native']
pretty_print_data(delta_means_rel)

3b+:
	go: 4570250.479177778
	js: 8247462.980244445
	py: 3261947.1746444446
	native: 42683.48628888889
3b.:
	go: 5151524.89388
	js: 8958540.465733333
	py: 3575884.898853333
	native: 49824.13672888889
3b+:
	go: 107.073036355221
	js: 193.22374288792278
	py: 76.42176069140761
3b.:
	go: 103.39416259053934
	js: 179.8032249806953
	py: 71.77013258274827


## Data time standard deviations (ns)

In [34]:

time_field = 'time (ns)'
runtime_device_data_stds = {}
for device in runtime_device_data.keys():
    runtime_device_data_stds[device] = {}
    for runtime in runtime_device_data[device].keys():
        runtime_device_data_stds[device][runtime] = runtime_device_data[device][runtime][time_field].std()
pretty_print_data(runtime_device_data_stds, 'ms')

3b+:
	go: 2.524160953178403
	js: 5.81342464970086
	py: 1.4528913005266697
	native: 0.014889705254302422
3b.:
	go: 2.845189445467819
	js: 6.368635902409733
	py: 1.5940647578133609
	native: 0.0182328473309366


In [25]:
# time.apply(lambda date: date/1000000).max()

0.39218